In [1]:
import pandas as pd
import numpy as np
from src.data_loader import *
from src.strategy import *
from src.index import *
from src.ticker_selection import *

In [2]:
data = DataGenerator('db').data_search()

7332117   1992-12-08
723782    1992-12-08
5261104   1992-12-08
5400948   1992-12-08
3377839   1992-12-08
             ...    
8394798   2022-11-01
9636650   2022-11-01
8394422   2022-11-01
9637684   2022-11-01
3516402   2022-11-01
Name: Datetime, Length: 11036522, dtype: datetime64[ns]

In [ ]:
def add_feature(df, **kwargs):
    data = df.copy()
    data["rsi"] = rsi(data)
    data[["macd","macd_signal","macd_oscillator"]] = macd(data, **kwargs)
    data[["en_center","en_ub","en_lb"]]            = envelope(data, **kwargs)
    data[["bo_center","bo_ub","bo_lb"]]            = bollinger(data, **kwargs)
    data[["slow_k","slow_d"]]                      = stochastic(data, **kwargs)
    data["sma(5)"]                                 = sma(data, 5)
    data["sma(20)"]                                = sma(data, 20) 
    data["sma(60)"]                                = sma(data, 60) 
    data["sma(120)"]                               = sma(data, 120)
    # data["sar"]                                    = sar(data, **kwargs)
    return data

In [ ]:
new_aapl = add_feature(aapl).dropna(axis = 0)

In [ ]:
new_aapl

In [ ]:
selected_aapl = stock_standard(aapl).calculator(9)

In [ ]:
selected_aapl

In [ ]:
rsi_strategy(aapl, 70, 30, a=1/28, w=28)['rsi_position'].value_counts()

In [ ]:
rsi_strategy(aapl, 70, 30)['rsi_position'].value_counts()

In [ ]:
macd_strategy(aapl, short=14, long=28, signal=10)['macd_position'].value_counts()

In [ ]:
envelope_strategy(aapl)['envelope_position'].value_counts()

In [ ]:
bollinger_strategy(aapl, w=30, k=4)['bollinger_position'].value_counts()

In [ ]:
stochastic_strategy(aapl)['stochastic_position'].value_counts()

In [ ]:
sell_sum(df = stochastic_strategy(aapl), ticker = 'AAPL', date = '2022-10-05', 'stochastic')

In [ ]:
data

In [ ]:
data['Ticker'][:3]

In [23]:
from tqdm import tqdm

In [24]:
data_list = []
for TICKER in tqdm(data['Ticker'].unique()):
    _ = DataGenerator('db').data_search(TICKER).sort_index()
    _ = rsi_strategy(_, 70, 30)
    _ = macd_strategy(_)
    _ = envelope_strategy(_)
    _ = bollinger_strategy(_)
    _ = stochastic_strategy(_)
    _['Ticker'] = TICKER
    data_list.append(_)
pd.concat(data_list, axis = 0)

100%|██████████| 4439/4439 [10:13:42<00:00,  8.30s/it]    


,Open,High,Low,Close,Adj Close,Volume,RSI,rsi_position,macd,macd_signal,...,en_lb,envelope_position,bo_center,bo_ub,bo_lb,bollinger_position,slow_k,slow_d,stochastic_position,Ticker
Datetime,,,,,,,,,,,,,,,,,,,,,
2004-08-19,2.502503,2.604104,2.401401,2.511011,2.511011,893181924.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-20,2.527778,2.729730,2.515015,2.710460,2.710460,456686856.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-23,2.771522,2.839840,2.728979,2.737738,2.737738,365122512.0,NaN,0,0.01,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-24,2.783784,2.792793,2.591842,2.624374,2.624374,304946748.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-25,2.626627,2.702703,2.599600,2.652653,2.652653,183772044.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-26,96.760002,98.540001,94.570000,94.820000,94.820000,71504300.0,37.731047,0,-1.05,-1.44,...,99.81460,1,99.956001,104.779030,95.132971,1,62.22,72.73,1,GOOG
2022-10-27,94.309998,95.169998,91.900002,92.599998,92.599998,54036500.0,34.892097,0,-1.59,-1.47,...,99.28792,1,99.681501,105.433542,93.929460,1,35.31,60.70,1,GOOG
2022-10-28,92.529999,96.860001,92.322998,96.580002,96.580002,35696900.0,43.150572,0,-1.68,-1.51,...,98.82660,1,99.703001,105.405078,94.000923,0,14.37,37.30,1,GOOG


In [ ]:
buy_sum(aapl, ticker = 'AAPL', date = '2022-10-05')

In [19]:
data_list[0]

,Open,High,Low,Close,Adj Close,Volume,RSI,rsi_position,macd,macd_signal,...,en_lb,envelope_position,bo_center,bo_ub,bo_lb,bollinger_position,slow_k,slow_d,stochastic_position,Ticker
Datetime,,,,,,,,,,,,,,,,,,,,,
2004-08-19,2.502503,2.604104,2.401401,2.511011,2.511011,893181924.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-20,2.527778,2.729730,2.515015,2.710460,2.710460,456686856.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-23,2.771522,2.839840,2.728979,2.737738,2.737738,365122512.0,NaN,0,0.01,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-24,2.783784,2.792793,2.591842,2.624374,2.624374,304946748.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
2004-08-25,2.626627,2.702703,2.599600,2.652653,2.652653,183772044.0,NaN,0,0.00,0.00,...,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0,GOOGL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-26,96.430000,98.309998,94.690002,94.930000,94.930000,88114100.0,38.926063,0,-0.93,-1.38,...,99.09735,1,99.324999,104.060953,94.589045,0,63.85,75.27,1,GOOGL
2022-10-27,94.519997,95.430000,91.800003,92.220001,92.220001,60710900.0,35.372752,0,-1.47,-1.40,...,98.57808,1,99.064999,104.680091,93.449908,1,35.08,61.93,1,GOOGL
2022-10-28,92.269997,96.550003,92.129997,96.290001,96.290001,43697300.0,43.686806,0,-1.55,-1.43,...,98.12436,1,99.096999,104.640720,93.553279,0,14.33,37.75,1,GOOGL
